In [17]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate(); Pkg.add("PyPlot"); Pkg.add("ForwardDiff")

  Activating environment at `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/lecture-notebooks/Lecture 10/Project.toml`
   Resolving package versions...
  No Changes to `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/lecture-notebooks/Lecture 10/Project.toml`
  No Changes to `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/lecture-notebooks/Lecture 10/Manifest.toml`
   Resolving package versions...
    Updating `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/lecture-notebooks/Lecture 10/Project.toml`
  [f6369f11] + ForwardDiff v0.10.35
    Updating `~/Dropbox/My Mac (MacBook Pro (2))/Desktop/CMU/Optimal Control/lecture-notebooks/Lecture 10/Manifest.toml`
  [f6369f11] ↑ ForwardDiff v0.10.34 ⇒ v0.10.35


In [15]:
using LinearAlgebra
using PyPlot
using ForwardDiff
using RobotZoo
using RobotDynamics
using MatrixCalculus

[ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
[ Info: Installing matplotlib via the Conda matplotlib package...
[ Info: Running `conda install -y matplotlib` in root environment


Solving environment: ...working... done

## Package Plan ##

  environment location: /Users/judsonvankyle/.julia/conda/3

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-1.0.9               |       hb7f2c08_8          19 KB  conda-forge
    brotli-bin-1.0.9           |       hb7f2c08_8          17 KB  conda-forge
    contourpy-1.0.7            |  py310ha23aa8a_0         204 KB  conda-forge
    cycler-0.11.0              |     pyhd8ed1ab_0          10 KB  conda-forge
    fonttools-4.39.3           |  py310h90acd4f_0         2.0 MB  conda-forge
    freetype-2.12.1            |       h3f81eb7_1         586 KB  conda-forge
    kiwisolver-1.4.4           |  py310ha23aa8a_1          63 KB  conda-forge
    lcms2-2.15                 |       h29502cd_0         222 KB  conda-forge
    lerc-4.0.0                 |       hb486fe8_0         284 KB  

kiwisolver-1.4.4     | 63 KB     | #########1                           |  25% 






xorg-libxdmcp-1.1.3  | 17 KB     | #################################### | 100% 









openjpeg-2.5.0       | 322 KB    | #7                                   |   5% 







xorg-libxau-1.0.9    | 11 KB     | #################################### | 100% 










lcms2-2.15           | 222 KB    | ##5                                  |   7% 


fonttools-4.39.3     | 2.0 MB    | ###################################9 | 100% 



unicodedata2-15.0.0  | 486 KB    | #################################### | 100% 



unicodedata2-15.0.0  | 486 KB    | #################################### | 100% 




pillow-9.4.0         | 44.1 MB   | ##4                                  |   7% 












pthread-stubs-0.4    | 6 KB      | #################################### | 100% 











munkres-1.1.4        | 12 KB     | #################################### | 100% 









openjpeg-2.5.0       | 322 KB    | #########

LoadError: ArgumentError: Package ForwardDiff not found in current path:
- Run `import Pkg; Pkg.add("ForwardDiff")` to install the ForwardDiff package.


In [16]:
#Acrobot Dynamics
a = RobotZoo.Acrobot()
h = 0.05

LoadError: UndefVarError: RobotZoo not defined

In [4]:
function dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = RobotDynamics.dynamics(a, x, u)
    f2 = RobotDynamics.dynamics(a, x + 0.5*h*f1, u)
    f3 = RobotDynamics.dynamics(a, x + 0.5*h*f2, u)
    f4 = RobotDynamics.dynamics(a, x + h*f3, u)
    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
end

dynamics_rk4 (generic function with 1 method)

In [5]:
function dfdx(x,u)
    ForwardDiff.jacobian(dx->dynamics_rk4(dx,u),x)
end

function dfdu(x,u)
    ForwardDiff.derivative(du->dynamics_rk4(x,du),u)
end

dfdu (generic function with 1 method)

In [6]:
Nx = 4     # number of state
Nu = 1     # number of controls
Tfinal = 10.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

LoadError: UndefVarError: h not defined

In [7]:
# Cost weights
Q = Diagonal([1.0*ones(2); 0.1*ones(2)]);
R = 0.01;
Qn = Array(100.0*I(Nx));

In [8]:
function stage_cost(x,u)
    return 0.5*((x-xgoal)'*Q*(x-xgoal)) + 0.5*R*u*u
end

stage_cost (generic function with 1 method)

In [9]:
function terminal_cost(x)
    return 0.5*(x-xgoal)'*Qn*(x-xgoal)
end

terminal_cost (generic function with 1 method)

In [10]:
function cost(xtraj,utraj)
    J = 0.0
    for k = 1:(Nt-1)
        J += stage_cost(xtraj[:,k],utraj[k])
    end
    J += terminal_cost(xtraj[:,Nt])
    return J
end

cost (generic function with 1 method)

In [11]:
#Initial guess
x0 = [-pi/2; 0; 0; 0]
xgoal = [pi/2; 0; 0; 0]
xtraj = kron(ones(1,Nt), x0)
utraj = randn(Nt-1);

LoadError: UndefVarError: Nt not defined

In [ ]:
#Initial Rollout
for k = 1:(Nt-1)
    xtraj[:,k+1] .= dynamics_rk4(xtraj[:,k],utraj[k])
end
J = cost(xtraj,utraj)

In [ ]:
#DDP Algorithm
p = zeros(Nx,Nt)
P = zeros(Nx,Nx,Nt)
d = ones(Nt-1)
K = zeros(Nu,Nx,Nt-1)
ΔJ = 0.0

xn = zeros(Nx,Nt)
un = zeros(Nt-1)

gx = zeros(Nx)
gu = 0.0
Gxx = zeros(Nx,Nx)
Guu = 0.0
Gxu = zeros(Nx)
Gux = zeros(Nx)

iter = 0
while maximum(abs.(d[:])) > 1e-3
    iter += 1
    
    p = zeros(Nx,Nt)
    P = zeros(Nx,Nx,Nt)
    d = ones(Nt-1)
    K = zeros(Nu,Nx,Nt-1)
    ΔJ = 0.0

    p[:,Nt] = Qn*(xtraj[:,Nt]-xgoal)
    P[:,:,Nt] = Qn
    
    #Backward Pass
    for k = (Nt-1):-1:1
        #Calculate derivatives
        q = Q*(xtraj[:,k]-xgoal)
        r = R*utraj[k]
    
        A = dfdx(xtraj[:,k], utraj[k]) #ForwardDiff.jacobian(dx->dynamics_rk4(dx,utraj[k]),xtraj[:,k])
        B = dfdu(xtraj[:,k], utraj[k]) #ForwardDiff.derivative(du->dynamics_rk4(xtraj[:,k],du),utraj[k])
    
        gx = q + A'*p[:,k+1]
        gu = r + B'*p[:,k+1]
    
        Gxx = Q + A'*P[:,:,k+1]*A 
        Guu = R + B'*P[:,:,k+1]*B 
        Gxu = A'*P[:,:,k+1]*B 
        Gux = B'*P[:,:,k+1]*A
        
        d[k] = Guu\gu
        K[:,:,k] .= Guu\Gux
    
        p[:,k] .= dropdims(gx - K[:,:,k]'*gu + K[:,:,k]'*Guu*d[k] - Gxu*d[k], dims=2)
        P[:,:,k] .= Gxx + K[:,:,k]'*Guu*K[:,:,k] - Gxu*K[:,:,k] - K[:,:,k]'*Gux
    
        ΔJ += gu'*d[k]
    end

    #Forward rollout with line search
    xn[:,1] = xtraj[:,1]
    α = 1.0

    for k = 1:(Nt-1)
        un[k] = utraj[k] - α*d[k] - dot(K[:,:,k],xn[:,k]-xtraj[:,k])
        xn[:,k+1] .= dynamics_rk4(xn[:,k],un[k])
    end
    Jn = cost(xn,un)
    
    while isnan(Jn) || Jn > (J - 1e-2*α*ΔJ)
        α = 0.5*α
        for k = 1:(Nt-1)
            un[k] = utraj[k] - α*d[k] - dot(K[:,:,k],xn[:,k]-xtraj[:,k])
            xn[:,k+1] .= dynamics_rk4(xn[:,k],un[k])
        end
        Jn = cost(xn,un)
    end
    #display(α)
    
    J = Jn
    xtraj .= xn
    utraj .= un
end

In [ ]:
iter

In [ ]:
J

In [ ]:
plot(thist,xtraj[1,:])
plot(thist,xtraj[2,:])

In [ ]:
plot(thist[1:Nt-1],utraj)

In [ ]:
using TrajOptPlots
using MeshCat
using StaticArrays

vis = Visualizer()
render(vis)

In [ ]:
TrajOptPlots.set_mesh!(vis, a)

In [ ]:
X1 = [SVector{4}(x) for x in eachcol(xtraj)];
visualize!(vis, a, thist[end], X1)